## wifi feature by kuto

[yukiさんのdataset](https://www.kaggle.com/dataset/951fe0fd675e58937311e936e941b01d517c227ce30192c8477bbe2bddeec602)をもとにを参考に、waypointを補正したwifi featureを作成する。  
trainにwifi_x, wifi_yが加わっただけ。  

v2は線形補完をtimestampを使ってやってみる(last seenは使っていないことに注意)

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import glob
import os
import sys 
sys.path.append("../")
from src.io_f import read_data_file

from scipy import interpolate

In [2]:
base_path = '..'
feature_dir = f"{base_path}/input/wifi-feature-with-timestamp"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

In [3]:
output_dir = "../input/kuto_wifi_dataset_v2/"
os.makedirs(output_dir, exist_ok=True)

In [4]:
train_file = train_files[0]
train = pd.read_csv(train_file, index_col=0).reset_index(drop=True).rename(columns={'foor_str':'floor_str'})
train

,000840e5c600de293cea57f13326f273c86c3988,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,...,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,timestamp,x,y,f,floor_str,path,time_diff
0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-53,-999,-999,1578462618392,230.037380,153.49635,-1,B1,5e15730aa280850006f3d005,434
1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-53,-999,-999,1578462618392,230.037380,153.49635,-1,B1,5e15730aa280850006f3d005,2331
2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-55,-999,-999,1578462618392,230.037380,153.49635,-1,B1,5e15730aa280850006f3d005,4257
3,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-52,-999,-999,1578462628512,231.402900,158.41515,-1,B1,5e15730aa280850006f3d005,-3938
4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-53,-999,-999,1578462628512,231.402900,158.41515,-1,B1,5e15730aa280850006f3d005,-2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9291,-999,-999,-999,-999,-999,-75,-999,-999,-86,-999,...,-999,-78,-999,1572951934638,123.473170,77.84787,3,F4,5dc1579ebce0a000068ae8eb,3771
9292,-999,-999,-999,-999,-999,-75,-999,-999,-84,-999,...,-999,-78,-999,1572951934638,123.473170,77.84787,3,F4,5dc1579ebce0a000068ae8eb,5801
9293,-999,-999,-999,-999,-999,-72,-999,-999,-82,-999,...,-999,-78,-999,1572951946278,126.059975,68.08712,3,F4,5dc1579ebce0a000068ae8eb,-3807
9294,-999,-999,-999,-999,-999,-72,-999,-999,-79,-999,...,-999,-999,-999,1572951946278,126.059975,68.08712,3,F4,5dc1579ebce0a000068ae8eb,-1777


In [5]:
train_files

['../input/wifi-feature-with-timestamp/5a0546857ecc773753327266_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5c3c44b80379370013e0fd2b_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d27075f03f801723c2e360f_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d27096c03f801723c31e5e0_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d27097f03f801723c320d97_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d27099f03f801723c32511d_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d2709a003f801723c3251bf_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d2709b303f801723c327472_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d2709bb03f801723c32852c_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d2709c303f801723c3299ee_timediff_1000_train.csv',
 '../input/wifi-feature-with-timestamp/5d2709d403f801723c32bd39_timediff_1000_train.csv',
 '../input

In [6]:
import sys 
sys.path.append("../")
from multiprocessing import Pool

def wifi_waypoint_by_linear_interpolation(
        trajectory_timestamp, wifi_timestamp,
        x_observed, y_observed, delta_time=1000,
        ): 

    wifi_waypoint_list = []
    num_interpolation = len(trajectory_timestamp) - 1  # 補完回数 
    # 各waypoint間で線形補完
    for i in range(num_interpolation):
        # 潜在的なtimestampを作成(これのどれかにwifiを当てはめるような形)
        n_split = int((max(trajectory_timestamp[i:i+2]) - min(trajectory_timestamp[i:i+2])) / delta_time) + 2  # delta_time刻みとなるように分割数を指定(+2は始点と終点分)
        timestamp_latent = np.linspace(min(trajectory_timestamp[i:i+2]), max(trajectory_timestamp[i:i+2]), n_split).astype(int) 
        
        # xが昇順の場合はlatentも昇順になるようにする
        if x_observed[i] < x_observed[i+1]:
            x_latent = np.linspace(min(x_observed[i:i+2]), max(x_observed[i:i+2]), n_split)
        # xが降順の場合はlatentも降順になるようにする
        else:
            x_latent = np.linspace(min(x_observed[i:i+2]), max(x_observed[i:i+2]), n_split)[::-1]
        
        # 線形補完関数の適用
        fitted_curve = interpolate.interp1d(x_observed[i:i+2], y_observed[i:i+2])

        # wifiのtimestampに最も近いものをsplit_timestampから取得しそれに対応するwaypointをwifiのwaypointとして取得
        wifi_x = []
        wifi_y = []

        # 区間内のwifiデータのみ考える
        if i == num_interpolation-1:
            # 最後のwaypointより未来のwifiデータがあるようなのでこの処理を追加
            target_idx = min(trajectory_timestamp[i:i+2]) <= wifi_timestamp
        else:
            target_idx = (min(trajectory_timestamp[i:i+2]) <= wifi_timestamp) & (wifi_timestamp < max(trajectory_timestamp[i:i+2]))
        target_wifi_timestamp = wifi_timestamp[target_idx]

        # timestampが最も近いものをwifiのwaypointとして取得
        for t in target_wifi_timestamp:
            idx = np.abs(timestamp_latent - t).argmin()  # wifiデータとtimestampが最も近いものをlatentから取得
            wifi_x.append(x_latent[idx])
            wifi_y.append(fitted_curve(x_latent[idx]))

        wifi_waypoint = np.stack([wifi_x, wifi_y], axis=1)
        wifi_waypoint_list.append(wifi_waypoint)

    wifi_waypoint = np.concatenate(wifi_waypoint_list)
    return wifi_waypoint

def get_wifi_waypoint(site, floor, path, timestamp):
    path_file = f'../input/indoor-location-navigation/train/{site}/{floor}/{path}.txt'
    example = read_data_file(path_file)

    # waypoint
    trajectory = example.waypoint
    trajectory_timestamp = trajectory[:,0]
    trajectory_waypoint = trajectory[:, 1:]
    x_observed = trajectory_waypoint[:, 0]
    y_observed = trajectory_waypoint[:, 1]

    # wifi
    wifi = example.wifi
    wifi_timestamp = np.unique(wifi[:, 0]).astype(int)

    # 線形補完
    wifi_waypoint = wifi_waypoint_by_linear_interpolation(trajectory_timestamp, wifi_timestamp,x_observed, y_observed) 
    return wifi_waypoint

In [7]:

def make_dataset_for_train(train_file):
    train = pd.read_csv(train_file, index_col=0).reset_index(drop=True).rename(columns={'foor_str':'floor_str'})
    file_name = train_file.split('/')[-1]
    num_of_lines = train.shape[0]
    print(f'{file_name} : {num_of_lines}')

    data_list = []
    for path, df in tqdm(train.groupby('path')):

        bssid = df.columns[:-7].values
        rssi = df.iloc[:, :-7].values
        targets = df.iloc[:,-7:].values

        # path内の各waypointのwifi値を取得
        sort_rssi = []
        sort_bssid = []
        for i in range(len(df)):
            sort_rssi.append(np.sort(rssi[i])[::-1][:100])
            idx = np.argsort(rssi[i])[::-1]
            sort_bssid.append(bssid[idx][:100])

        sort_rssi = np.stack(sort_rssi)
        sort_bssid = np.stack(sort_bssid)
        site = train_file.split('/')[-1].split('_')[0]
        timestamp = targets[i][0]
        floor = targets[i][4]
  
        wifi_waypoint = get_wifi_waypoint(site, floor, path, timestamp)  # 1行ずつ処理するのは計算コスト的にもったいない
        path_data = np.concatenate((sort_bssid, sort_rssi, targets, wifi_waypoint), axis=1)
        path_df = pd.DataFrame(path_data)
        data_list.append(path_df)
    
    data_df = pd.concat(data_list)
    columns = [f'bssid_{str(i)}' for i in range(100)] + [f'rssi_{str(i)}' for i in range(100)] + \
            ['timestamp', 'x', 'y', 'floor', 'floor_str', 'path', 'time_diff', 'wifi_x', 'wifi_y']
    data_df.columns = columns
    data_df.to_csv(output_dir + file_name, index=False)


In [8]:
with Pool(processes=24) as pool:
    pool.map(make_dataset_for_train, train_files)

5da138274db8ce0c98bbd3d2_timediff_1000_train.csv : 2662
5da138364db8ce0c98bc00f1_timediff_1000_train.csv : 2767


5da1389e4db8ce0c98bd0547_timediff_1000_train.csv : 6312


5d27099f03f801723c32511d_timediff_1000_train.csv : 4251
5d2709a003f801723c3251bf_timediff_1000_train.csv : 3940


5da138754db8ce0c98bca82f_timediff_1000_train.csv : 7188


5d27096c03f801723c31e5e0_timediff_1000_train.csv : 9100


...

5dc8cea7659e181adb076a3f_timediff_1000_train.csv : 15655


5da138b74db8ce0c98bd4774_timediff_1000_train.csv : 17382


5dbc1d84c1eb61796cf7c010_timediff_1000_train.csv : 16174


5d2709c303f801723c3299ee_timediff_1000_train.csv : 10083




5d27075f03f801723c2e360f_timediff_1000_train.csv : 23666


In [15]:
def make_dataset_for_test(test_file):
    train = pd.read_csv(test_file)

    file_name = test_file.split('/')[-1]

    num_of_lines = train.shape[0]
    print(f'{file_name} : {num_of_lines}')

    data = None
    for i in tqdm(range(num_of_lines)):

        tmp = train.iloc[i,1:-2].astype(int).sort_values(ascending=False).head(100)
        target = train.iloc[i, -2:]

        line = pd.concat([pd.Series(tmp.index.astype(str)), tmp.astype(int), pd.Series(target)])
        line.index = [str(i) for i in range(202)]
        if data is None:
            data = pd.DataFrame(line).T
            data.columns = [str(i) for i in range(202)]
        else:
            data = data.append(line, ignore_index=True)
    data.columns = [f'bssid_{str(i)}' for i in range(100)] + [f'rssi_{str(i)}' for i in range(100)] + ['site_path_timestamp', 'time_diff']
    data.to_csv(output_dir + file_name, index=False)

In [17]:
with Pool(processes=24) as pool:
    pool.map(make_dataset_for_test, test_files)

5da138274db8ce0c98bbd3d2_timediff_1000_test.csv : 103
5da1389e4db8ce0c98bd0547_timediff_1000_test.csv : 1745d27099f03f801723c32511d_timediff_1000_test.csv : 49

5da138364db8ce0c98bc00f1_timediff_1000_test.csv : 139


5d2709a003f801723c3251bf_timediff_1000_test.csv : 218
5da138314db8ce0c98bbf3a0_timediff_1000_test.csv : 171


5da138754db8ce0c98bca82f_timediff_1000_test.csv : 386
5d27096c03f801723c31e5e0_timediff_1000_test.csv : 654
5d2709e003f801723c32d896_timediff_1000_test.csv : 531


5da1383b4db8ce0c98bc11ab_timediff_1000_test.csv : 380


5d27097f03f801723c320d97_timediff_1000_test.csv : 303
5da138764db8ce0c98bcaa46_timediff_1000_test.csv : 573
5c3c44b80379370013e0fd2b_timediff_1000_test.csv : 26


...

5d2709bb03f801723c32852c_timediff_1000_test.csv : 716
5a0546857ecc773753327266_timediff_1000_test.csv : 299
5d27075f03f801723c2e360f_timediff_1000_test.csv : 47


5da138b74db8ce0c98bd4774_timediff_1000_test.csv : 445


5dc8cea7659e181adb076a3f_timediff_1000_test.csv : 648


5da958dd46f8266d0737457b_timediff_1000_test.csv : 778


5dbc1d84c1eb61796cf7c010_timediff_1000_test.csv : 923
5d2709c303f801723c3299ee_timediff_1000_test.csv : 509


In [19]:
import pickle
dfs = []
for f in glob.glob(output_dir + '*train.csv'):
    site_id = f.split('/')[-1].split('_')[0]
    _df = pd.read_csv(f)
    _df['site_id'] = site_id
    dfs.append(_df)

In [20]:
def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
to_pickle(output_dir + 'train_all.pkl', pd.concat(dfs).reset_index(drop=True))

In [21]:
dfs = []
for f in glob.glob(output_dir + '*test.csv'):
    site_id = f.split('/')[-1].split('_')[0]
    _df = pd.read_csv(f)
    _df['site_id'] = site_id
    dfs.append(_df)

In [22]:
to_pickle(output_dir + 'test_all.pkl', pd.concat(dfs).reset_index(drop=True))